In [36]:
import requests
import ast

In [37]:
url = 'http://www.omdbapi.com/?t=The+Matrix&apikey=f04aea9f'

In [38]:
response = requests.get(url)

In [41]:
imdb_dict = ast.literal_eval(response.content.decode("UTF-8"))

In [44]:
imdb_dict['imdbRating']

'8.7'

In [45]:
imdb_dict['Metascore']

'73'

In [43]:
imdb_dict

{'Title': 'The Matrix',
 'Year': '1999',
 'Rated': 'R',
 'Released': '31 Mar 1999',
 'Runtime': '136 min',
 'Genre': 'Action, Sci-Fi',
 'Director': 'Lana Wachowski, Lilly Wachowski',
 'Writer': 'Lilly Wachowski, Lana Wachowski',
 'Actors': 'Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving',
 'Plot': 'When a beautiful stranger leads computer hacker Neo to a forbidding underworld, he discovers the shocking truth--the life he knows is the elaborate deception of an evil cyber-intelligence.',
 'Language': 'English',
 'Country': 'USA',
 'Awards': 'Won 4 Oscars. Another 38 wins & 51 nominations.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BNzQzOTk3OTAtNDQ0Zi00ZTVkLWI0MTEtMDllZjNkYzNjNTc4L2ltYWdlXkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.7/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '88%'},
  {'Source': 'Metacritic', 'Value': '73/100'}],
 'Metascore': '73',
 'imdbRating': '8.7',
 'imdbVotes': '1,702,96

In [1]:
import pandas as pd
import conf
import os

In [2]:
movies = pd.read_csv('output.csv')

In [3]:
movies.head()

,url,title,genre,length,audience_score,tomato_score,year,poster,desc
0,https://www.rottentomatoes.com/m/on_body_and_s...,On Body and Soul (A Teströl és Lélekröl),Drama,1h 56m,82.0,NaN,2017,https://resizing.flixster.com/MMwhnlo0d5Q6g-zq...,When a man and woman who meet at work begin to...
1,https://www.rottentomatoes.com/m/always_be_my_...,Always Be My Maybe,Romance,1h 42m,80.0,90.0,2019,https://resizing.flixster.com/l-FdhverVu-cPJ3y...,Childhood sweethearts have a falling out and d...
2,https://www.rottentomatoes.com/m/the_kindergar...,The Kindergarten Teacher,Drama,1h 36m,68.0,90.0,2018,https://resizing.flixster.com/ZFQ5JbHf9ktwFNXB...,A teacher sees such great promise in her 5-yea...
3,https://www.rottentomatoes.com/m/departed/,The Departed,Crime/Mystery and thriller,2h 32m,94.0,90.0,2006,https://resizing.flixster.com/cpZ3WiuL4SQODLSN...,South Boston cop Billy Costigan (Leonardo DiCa...
4,https://www.rottentomatoes.com/m/1922_2017/,1922,Horror,1h 42m,57.0,91.0,2017,https://resizing.flixster.com/kBMRd_diQ5VYuIGL...,A rancher conspires to murder his wife for fin...


In [5]:
from decouple import config

In [33]:
import time
import grequests
import re
import string

In [34]:
def get_responses_from_urls(urls):
    """receives urls and returns a dict:
    keys = urls
    values = responses"""
    try:
        # in order to prevent the site from blocking us: sleep 1 sec
        time.sleep(1)
        rs = (grequests.get(url) for url in urls)
        # If the response was successful, no Exception will be raised
        responses = grequests.map(rs)
        responses_dict = {}
        for i, url in enumerate(urls):
            responses_dict[url] = responses[i]
    except Exception as err:
        sys.exit(1)
    return responses_dict

In [49]:
def create_omdb_url(title):
    key = config('API_KEY')
    url = f'http://www.omdbapi.com/?t={title}&apikey={key}'
    url=re.sub("\(.*?\)","()",url)\
          .replace('(', '')\
          .replace(')', '')\
          .rstrip()\
          .replace(' ', '_')
    return url

In [50]:
for x in range(int(len(movies) / conf.BATCH_SIZE)):
    # runs from 0 till len(movies) / BATCH_SIZE
    idxs = movies.index[x * conf.BATCH_SIZE:x * conf.BATCH_SIZE + conf.BATCH_SIZE]
    titles = movies[movies.index.isin(idxs)]['title']
    
    urls = [create_omdb_url(t) for t in titles]
    responses_dict = get_responses_from_urls(urls)
    print(responses_dict)
    break

{'http://www.omdbapi.com/?t=On_Body_and_Soul_&apikey=f04aea9f': <Response [200]>, 'http://www.omdbapi.com/?t=Always_Be_My_Maybe&apikey=f04aea9f': <Response [200]>, 'http://www.omdbapi.com/?t=The_Kindergarten_Teacher&apikey=f04aea9f': <Response [200]>, 'http://www.omdbapi.com/?t=The_Departed&apikey=f04aea9f': <Response [200]>, 'http://www.omdbapi.com/?t=1922&apikey=f04aea9f': <Response [200]>, 'http://www.omdbapi.com/?t=Uncorked&apikey=f04aea9f': <Response [200]>, 'http://www.omdbapi.com/?t=The_Willoughbys&apikey=f04aea9f': <Response [200]>, 'http://www.omdbapi.com/?t=I_Am_Mother&apikey=f04aea9f': <Response [200]>, "http://www.omdbapi.com/?t=Gerald's_Game&apikey=f04aea9f": <Response [200]>, 'http://www.omdbapi.com/?t=Happy_as_Lazzaro_&apikey=f04aea9f': <Response [200]>}


In [51]:
list(responses_dict.keys())

['http://www.omdbapi.com/?t=On_Body_and_Soul_&apikey=f04aea9f',
 'http://www.omdbapi.com/?t=Always_Be_My_Maybe&apikey=f04aea9f',
 'http://www.omdbapi.com/?t=The_Kindergarten_Teacher&apikey=f04aea9f',
 'http://www.omdbapi.com/?t=The_Departed&apikey=f04aea9f',
 'http://www.omdbapi.com/?t=1922&apikey=f04aea9f',
 'http://www.omdbapi.com/?t=Uncorked&apikey=f04aea9f',
 'http://www.omdbapi.com/?t=The_Willoughbys&apikey=f04aea9f',
 'http://www.omdbapi.com/?t=I_Am_Mother&apikey=f04aea9f',
 "http://www.omdbapi.com/?t=Gerald's_Game&apikey=f04aea9f",
 'http://www.omdbapi.com/?t=Happy_as_Lazzaro_&apikey=f04aea9f']